In [3]:
!sudo pip install transformers --upgrade

     |████████████████████████████████| 7.8MB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 47.5MB/s eta 0:00:01
  Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [1]:
import torch
from transformers import pipeline

#pipe = pipeline("text-generation", model="HuggingFaceH4/starchat-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-8/oct-30/merged-model", torch_dtype=torch.float16, device_map="cpu")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: Can't load tokenizer for '/data/mistral/query-to-mql/exp-8/oct-30/merged-model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/data/mistral/query-to-mql/exp-8/oct-30/merged-model' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt = prompt_template.format(query="How do I sort a list in Python?")
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)

t2 = time.time()
t2-t1

In [ ]:
outputs[0]["generated_text"].split("\n")